#Setup

Installs 

In [ ]:
!git clone --recursive https://github.com/ahmetumutdurmus/awd-lstm.git

In [ ]:
!pip install python-Levenshtein

In [ ]:
!git clone --recursive https://github.com/PetrochukM/PyTorch-NLP.git
%cd PyTorch-NLP
!pip install .
%cd ..

In [ ]:
!git clone --recursive https://github.com/parlance/ctcdecode.git
!pip install wget
%cd ctcdecode
!pip install .
%cd ..

In [ ]:
!pip install wandb --upgrade

Imports

In [ ]:
cd awd-lstm

/content/awd-lstm


In [ ]:
import numpy as np
import torch 
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision   
import torch.nn as nn
import torch.nn.functional as F
import json
import pandas as pd
from datetime import datetime
import pdb 
import pandas as pd
from psutil import virtual_memory
import os
from zipfile import ZipFile
import matplotlib.pyplot as plt
from PIL import Image
import random
import shutil
from ctcdecode import CTCBeamDecoder
from Levenshtein import distance
import torchaudio
from google.colab import drive
from collections import OrderedDict
from torchnlp.nn import WeightDropLSTM
from ntasgd import NTASGD
from os import listdir
import wandb

In [ ]:
cd /content

/content


In [ ]:
class SetUpColab():

  def __init__(self):
    pass
    
  #Determines how much ram the runtime has
  @staticmethod
  def runtime_info():
    ram_gb = virtual_memory().total / 1e9
    print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))
    if ram_gb < 20:
      print('Not using a high-RAM runtime')
    else:
      print('You are using a high-RAM runtime!')
    !nvidia-smi
    
  @staticmethod
  def mount_google_drive():
    drive._mount('/content/gdrive')
  
  #Sets up environement for use with kaggle api
  @staticmethod
  def set_up_kaggle():
    !pip uninstall -y kaggle
    !pip install --upgrade pip
    !pip install kaggle==1.5.6
    !mkdir .kaggle

    token = {"username":"nicholasmagal","key":"9bf671834d75b58fac2b037da15f4cf0"}
    with open('/content/.kaggle/kaggle.json', 'w') as file:
      json.dump(token, file)
    
    for i in range(2):
      !chmod 600 /content/.kaggle/kaggle.json
      !cp /content/.kaggle/kaggle.json /root/.kaggle/
      !kaggle config set -n path -v /content

  @staticmethod
  def setup_wandb():
    wandb.login()

  #Calls above methods to do a complete Collab setup, ready to run ml models :D Note may want to change this per competition
  @staticmethod
  def complete_set_up():
    SetUpColab.runtime_info()
    SetUpColab.mount_google_drive()
    SetUpColab.set_up_kaggle()
    SetUpColab.setup_wandb()
  

In [ ]:
SetUpColab.complete_set_up()
!kaggle competitions download -c 11785-fall2021-hw3p2

In [ ]:
%cd . 
!unzip /content/competitions/11785-fall2021-hw3p2/11785-fall2021-hw3p2.zip

#Data Setup

In [ ]:
def create_data_loader(dataset, shuffle, batch_size = 128, num_workers = 1, pin_memory = True, collate_fn = None):
  data_loader = torch.utils.data.DataLoader(dataset=dataset,
                                                  batch_size = batch_size,
                                                  shuffle = shuffle,
                                                  num_workers = num_workers,
                                                  pin_memory = pin_memory,
                                                  collate_fn = collate_fn)
  return data_loader

In [ ]:
# Define dataset class
class UtteranceDatset(Dataset):
  # load the dataset
  def __init__(self, x_path, y_path):
    self.X = np.load(x_path, allow_pickle=True)
    self.Y = np.load(y_path, allow_pickle=True)

  # get number of items/rows in dataset
  def __len__(self):
    return len(self.Y)

  # get row item at some index
  def __getitem__(self, index):
    x = torch.FloatTensor(self.X[index])
    y = torch.LongTensor(self.Y[index])

    return x, y

  def collate_fn(batch):
    features, labels = zip(*batch)
    features_length = [len(x) for x in features]
    labels_length =[len(y) for y in labels]

    features = torch.nn.utils.rnn.pad_sequence(features, batch_first = True)
    labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first = True)

    #Features returned as B X L X Frequency Bins
    #Labels returned as B X L
    return(features, labels, features_length, labels_length)

In [ ]:
# Define dataset class
class UtteranceDatset_Test(Dataset):
  # load the dataset
  def __init__(self, x_path):
    self.X = np.load(x_path, allow_pickle=True)

  # get number of items/rows in dataset
  def __len__(self):
    return len(self.X)

  # get row item at some index
  def __getitem__(self, index):
    x = torch.FloatTensor(self.X[index])
    return x

  def collate_fn(batch):
    features_length = [len(x) for x in batch]
    features = torch.nn.utils.rnn.pad_sequence(batch, batch_first = True)

    #Features returned as B X L X Frequency Bins
    #Labels returned as B X L
    return(features, features_length)

#Models

In [ ]:
class GRUModelConv(nn.Module):

  def __init__(self, hidden_size, num_layers, dropout):
    super(GRUModelConv, self).__init__()
    self.convolution_embedding = nn.Sequential(nn.Conv1d(in_channels = 40, out_channels = 256, kernel_size=3, bias = True, padding='same'), nn.ReLU(), nn.BatchNorm1d(256),nn.Conv1d(in_channels = 256, out_channels = 256, kernel_size=3, bias = False, padding='same'), nn.ReLU(), nn.BatchNorm1d(256),nn.Dropout(p=dropout))
    self.biGRU_0 = nn.GRU(input_size=256, hidden_size=hidden_size, num_layers=num_layers, bidirectional=True, dropout = dropout, batch_first = True)
    self.biGRU_1 = nn.GRU(input_size=hidden_size*2, hidden_size=hidden_size, num_layers = num_layers, bidirectional=True, dropout = dropout, batch_first = True)
    self.linear_layers = nn.Sequential(nn.ReLU(), nn.Linear(2*hidden_size, 128, bias=False), nn.ReLU(), torch.nn.Dropout(), nn.Linear(128,42))
  def forward(self, x, lengths_data): 
    # x dimen =  B X L X C
    # CNN dim-Input = (B, C, L), output = (B, C, L)

    x = x.permute(0, 2, 1)
    #x = self.transforms(x)
    x = self.convolution_embedding(x)

    #LSTM Input = B x L X C, Output = B X L X Hidden * D 
    x = x.permute(0,2,1)
    x = torch.nn.utils.rnn.pack_padded_sequence(x, lengths_data, enforce_sorted=False, batch_first=True)

    x, state = self.biGRU_0(x)
    x, state = self.biGRU_1(x)
    x, length_1 = torch.nn.utils.rnn.pad_packed_sequence(x, batch_first=True)

    #Linear dim-Iput = * X H, dim-Output = B x L x C
    x = self.linear_layers(x)
    return(x,length_1)

In [ ]:
class LSTMModelConv(nn.Module):

  def __init__(self, hidden_size, num_layers, dropout):
    super(LSTMModelConv, self).__init__()
    #frequecy masking, timemasking, catuion noise, augmentations 
    #self.transforms = nn.Sequential(torchaudio.transforms.FrequencyMasking(freq_mask_param=15),torchaudio.transforms.TimeMasking(time_mask_param=15))
    self.convolution_embedding = nn.Sequential(nn.Conv1d(in_channels = 40, out_channels = 256, kernel_size=3, bias = False, padding='same'), nn.ReLU(), nn.BatchNorm1d(256),nn.Conv1d(in_channels = 256, out_channels = 256, kernel_size=3, bias = False, padding='same'), nn.ReLU(), nn.BatchNorm1d(256),nn.Dropout(p=dropout))
    self.biLSTM_0 = nn.LSTM(input_size=256, hidden_size=hidden_size, num_layers=num_layers, bidirectional=True, dropout = dropout, batch_first = True)
    self.biLSTM_1 = nn.LSTM(input_size=hidden_size*2, hidden_size=hidden_size, num_layers = num_layers, bidirectional=True, dropout = dropout, batch_first = True)
    self.linear_layers = nn.Sequential(nn.ReLU(), nn.Linear(2*hidden_size, 128, bias=False), nn.ReLU(), torch.nn.Dropout(), nn.Linear(128,42))
    
  def forward(self, x, lengths_data): 
    # x dimen =  B X L X C
    # CNN dim-Input = (B, C, L), output = (B, C, L)
    x = x.permute(0, 2, 1)
    #x = self.transforms(x)
    x = self.convolution_embedding(x)

    #LSTM Input = B x L X C, Output = B X L X Hidden * D 
    x = x.permute(0,2,1)
    x = torch.nn.utils.rnn.pack_padded_sequence(x, lengths_data, enforce_sorted=False, batch_first=True)

    x, state = self.biLSTM_0(x)
    x, state = self.biLSTM_1(x)
    x, length_1 = torch.nn.utils.rnn.pad_packed_sequence(x, batch_first=True)

    #Linear dim-Iput = * X H, dim-Output = B x L x C
    x = self.linear_layers(x)
    return(x,length_1)


In [ ]:
class LSTMModelConvModifiedBigNetwork(nn.Module):

  def __init__(self, hidden_size, num_layers, dropout):
    super(LSTMModelConvModifiedBigNetwork, self).__init__()
    #frequecy masking, timemasking, catuion noise, augmentations 
    #self.transforms = nn.Sequential(torchaudio.transforms.FrequencyMasking(freq_mask_param=15),torchaudio.transforms.TimeMasking(time_mask_param=15))
    self.convolution_embedding = nn.Sequential(nn.Conv1d(in_channels = 40, out_channels = 256, kernel_size=3, bias = False, padding='same'), nn.LeakyReLU(), nn.BatchNorm1d(256),nn.Conv1d(in_channels = 256, out_channels = 256, kernel_size=3, bias = False, padding='same'), nn.LeakyReLU(), nn.BatchNorm1d(256),nn.Dropout(p=.3))
    self.biLSTM_0 = nn.LSTM(input_size=256, hidden_size=hidden_size, num_layers=num_layers, bidirectional=False, dropout = dropout, batch_first = True)
    self.biLSTM_1 = nn.LSTM(input_size=hidden_size, hidden_size=hidden_size, num_layers = num_layers, bidirectional=True, dropout = dropout, batch_first = True)
    self.linear_layers = nn.Sequential(nn.LeakyReLU(), nn.Linear(2*hidden_size, 128, bias=False), nn.LeakyReLU(), torch.nn.Dropout(), nn.Linear(128,42))
  def forward(self, x, lengths_data): 
    # x dimen =  B X L X C
    # CNN dim-Input = (B, C, L), output = (B, C, L)

    x = x.permute(0, 2, 1)
    #x = self.transforms(x)
    #Transforms

    x = self.convolution_embedding(x)

    #LSTM Input = B x L X C, Output = B X L X Hidden * D 
    x = x.permute(0,2,1)
    x = torch.nn.utils.rnn.pack_padded_sequence(x, lengths_data, enforce_sorted=False, batch_first=True)

    x, state = self.biLSTM_0(x)
    x, state = self.biLSTM_1(x)
    x, length_1 = torch.nn.utils.rnn.pad_packed_sequence(x, batch_first=True)

    #Linear dim-Iput = * X H, dim-Output = B x L x C
    x = self.linear_layers(x)
    return(x,length_1)

In [ ]:
class LSTMModelVanilla(nn.Module):

  def __init__(self, hidden_size, num_layers, dropout):
    super(LSTMModelVanilla, self).__init__()

    self.biLSTM_0 = nn.LSTM(input_size=40, hidden_size=hidden_size, num_layers=num_layers, bidirectional=True, dropout = dropout, batch_first = True)
    self.biLSTM_1 = nn.LSTM(input_size=hidden_size*2, hidden_size=hidden_size, num_layers = num_layers, bidirectional=True, dropout = dropout, batch_first = True)
    self.linear_layers = nn.Sequential(nn.ReLU(), nn.Linear(2*hidden_size, 128, bias=False), nn.ReLU(), torch.nn.Dropout(), nn.Linear(128,42))
  def forward(self, x, lengths_data): 
    # x dimen =  B X L X C
    #LSTM Input = B x L X C, Output = B X L X Hidden * D 
    x = torch.nn.utils.rnn.pack_padded_sequence(x, lengths_data, enforce_sorted=False, batch_first=True)

    x, state = self.biLSTM_0(x)
    x, state = self.biLSTM_1(x)
    x, length_1 = torch.nn.utils.rnn.pad_packed_sequence(x, batch_first=True)

    #Linear dim-Iput = * X H, dim-Output = B x L x C
    x = self.linear_layers(x)
    return(x,length_1)

In [ ]:
class LSTMModelConvTransforms(nn.Module):

  def __init__(self, hidden_size, num_layers, dropout):
    super(LSTMModelConvTransforms, self).__init__()
    #frequecy masking, timemasking, catuion noise, augmentations 
    self.transforms = nn.Sequential(torchaudio.transforms.FrequencyMasking(freq_mask_param=15),torchaudio.transforms.TimeMasking(time_mask_param=100))
    self.convolution_embedding = nn.Sequential(nn.Conv1d(in_channels = 40, out_channels = 256, kernel_size=3, bias = False, padding='same'), nn.ReLU(), nn.BatchNorm1d(256),nn.Conv1d(in_channels = 256, out_channels = 256, kernel_size=3, bias = False, padding='same'), nn.ReLU(), nn.BatchNorm1d(256),nn.Dropout(p=.4))
    self.biLSTM_0 = nn.LSTM(input_size=256, hidden_size=hidden_size, num_layers=num_layers, bidirectional=True, dropout = dropout, batch_first = True)
    self.biLSTM_1 = nn.LSTM(input_size=hidden_size*2, hidden_size=hidden_size, num_layers = num_layers, bidirectional=True, dropout = dropout, batch_first = True)
    self.linear_layers = nn.Sequential(nn.ReLU(), nn.Linear(2*hidden_size, 128, bias=False), nn.ReLU(), torch.nn.Dropout(), nn.Linear(128,42))
  def forward(self, x, lengths_data): 
    # x dimen =  B X L X C
    # CNN dim-Input = (B, C, L), output = (B, C, L)

    x = x.permute(0, 2, 1)
    x = self.transforms(x)
    #Transforms
    x = self.convolution_embedding(x)

    #LSTM Input = B x L X C, Output = B X L X Hidden * D 
    x = x.permute(0,2,1)
    x = torch.nn.utils.rnn.pack_padded_sequence(x, lengths_data, enforce_sorted=False, batch_first=True)

    x, state = self.biLSTM_0(x)
    x, state = self.biLSTM_1(x)
    x, length_1 = torch.nn.utils.rnn.pad_packed_sequence(x, batch_first=True)

    #Linear dim-Iput = * X H, dim-Output = B x L x C
    x = self.linear_layers(x)
    return(x,length_1)

In [ ]:
class pBLSTM(nn.Module):

    def __init__(self, hidden_dim, layers_per_lstm,dropout):
        super(pBLSTM, self).__init__()

        self.convolution_embedding = nn.Sequential(nn.Conv1d(in_channels = 40, out_channels = 256, kernel_size=3, bias = True, padding='same'), nn.ReLU(), nn.BatchNorm1d(256),nn.Conv1d(in_channels = 256, out_channels = 256, kernel_size=3, bias = False, padding='same'), nn.ReLU(), nn.BatchNorm1d(256),nn.Dropout(p=dropout))

        self.blstm_0 = nn.LSTM(input_size = 256, hidden_size = hidden_dim, num_layers = layers_per_lstm, bidirectional = True, batch_first = True)
        self.blstm_1 = nn.LSTM(input_size = hidden_dim*2, hidden_size = hidden_dim, num_layers=layers_per_lstm,bidirectional = True, batch_first = True)
        #self.blstm_2 = nn.LSTM(input_size = hidden_dim*2, hidden_size = hidden_dim, num_layers=layers_per_lstm,bidirectional = True, batch_first = True)
        #self.blstm_3 = nn.LSTM(input_size = hidden_dim*2, hidden_size = hidden_dim, num_layers=layers_per_lstm,bidirectional = True, batch_first = True)

        self.dropout = nn.Dropout(p = dropout)
        self.linear_layers = nn.Sequential(nn.ReLU(), nn.Linear(hidden_dim*2, 128, bias=False), nn.ReLU(), torch.nn.Dropout(p = dropout), nn.Linear(128,42))

    def forward(self, x, lengths_data):
      # x dimen =  B X L X C
      # CNN dim-Input = (B, C, L), output = (B, C, L)
      x = x.permute(0, 2, 1)
      x = self.convolution_embedding(x)

      #LSTM Input = B x L X C, Output = B X L X Hidden * D 
      x = x.permute(0,2,1)
      x = torch.nn.utils.rnn.pack_padded_sequence(x, lengths_data, enforce_sorted=False, batch_first=True)
      x, state = self.blstm_0(x)     

      #First level of pLSTM
      x, batch_lengths = torch.nn.utils.rnn.pad_packed_sequence(x, batch_first=True)
      x = self.dropout(x)
      if x.shape[1]%2 !=0:
        x = x[:, 0:-1, :]
      batch_lengths = [lens//2 for lens in batch_lengths]
      x = torch.reshape(x, (x.shape[0], int(x.shape[1]/2), 2, x.shape[2]))
      x = torch.mean(x, dim = 2)
      x = torch.nn.utils.rnn.pack_padded_sequence(x, batch_lengths, enforce_sorted=False, batch_first=True)
      x, state = self.blstm_1(x)

      # #Second level of pLSTM
      # x, batch_lengths = torch.nn.utils.rnn.pad_packed_sequence(x, batch_first=True)  
      # x = self.dropout(x)
      # if x.shape[1]%2 !=0:
      #   x = x[:, 0:-1, :] 
      # batch_lengths = [lens//2 for lens in batch_lengths]
      # x = torch.reshape(x, (x.shape[0], int(x.shape[1]/2), 2, x.shape[2]))
      # x = torch.mean(x, dim = 2)
      # x = torch.nn.utils.rnn.pack_padded_sequence(x, batch_lengths, enforce_sorted=False, batch_first=True)
      # x, state = self.blstm_2(x)

      # #Third level of pLSTM - This causes NAN as output for training loss
      # x, batch_lengths = torch.nn.utils.rnn.pad_packed_sequence(x, batch_first=True)  
      # x = self.dropout(x)
      # if x.shape[1]%2 !=0:
      #   x = x[:, 0:-1, :] 
      # batch_lengths = [lens//2 for lens in batch_lengths]
      # x = torch.reshape(x, (x.shape[0], int(x.shape[1]/2), 2, x.shape[2]))
      # x = torch.mean(x, dim = 2)
      # x = torch.nn.utils.rnn.pack_padded_sequence(x, batch_lengths, enforce_sorted=False, batch_first=True)
      # x, state = self.blstm_3(x)
      
      #Linear dim-Iput = * X H, dim-Output = B x L x C
      x, batch_lengths = torch.nn.utils.rnn.pad_packed_sequence(x, batch_first=True)  
      x = self.linear_layers(x)
      return(x,batch_lengths)





In [ ]:
class LSTMModelConvDropConnect(nn.Module):

  def __init__(self, hidden_size, num_layers, dropout):
    super(LSTMModelConvDropConnect, self).__init__()
    #frequecy masking, timemasking, catuion noise, augmentations 
    #self.transforms = nn.Sequential(torchaudio.transforms.FrequencyMasking(freq_mask_param=15),torchaudio.transforms.TimeMasking(time_mask_param=15))
    self.convolution_embedding = nn.Sequential(nn.Conv1d(in_channels = 40, out_channels = 256, kernel_size=3, bias = False, padding='same'), nn.ReLU(), nn.BatchNorm1d(256),nn.Conv1d(in_channels = 256, out_channels = 256, kernel_size=3, bias = False, padding='same'), nn.ReLU(), nn.BatchNorm1d(256),nn.Dropout(p=dropout))
    self.biLSTM_0 = WeightDropLSTM(input_size=256, hidden_size=hidden_size, num_layers=num_layers, bidirectional=True, dropout = dropout, batch_first = True, weight_dropout = dropout)
    self.biLSTM_1 = WeightDropLSTM(input_size=hidden_size*2, hidden_size=hidden_size, num_layers = num_layers, bidirectional=True, dropout = dropout, batch_first = True, weight_dropout = dropout)
    self.linear_layers = nn.Sequential(nn.ReLU(), nn.Linear(2*hidden_size, 128, bias=False), nn.ReLU(), torch.nn.Dropout(), nn.Linear(128,42))
  def forward(self, x, lengths_data): 
    # x dimen =  B X L X C
    # CNN dim-Input = (B, C, L), output = (B, C, L)
  
    x = x.permute(0, 2, 1)
    #x = self.transforms(x)
    #Transforms
    x = self.convolution_embedding(x)

    #LSTM Input = B x L X C, Output = B X L X Hidden * D 
    x = x.permute(0,2,1)
    x = torch.nn.utils.rnn.pack_padded_sequence(x, lengths_data, enforce_sorted=False, batch_first=True)

    x, state = self.biLSTM_0(x)
    x, state = self.biLSTM_1(x)
    x, length_1 = torch.nn.utils.rnn.pad_packed_sequence(x, batch_first=True)

    #Linear dim-Iput = * X H, dim-Output = B x L x C
    x = self.linear_layers(x)
    return(x,length_1)

In [ ]:
class LSTMModelConvDropConnect2(nn.Module):

  def __init__(self, hidden_size, num_layers, dropout):
    super(LSTMModelConvDropConnect2, self).__init__()
    #frequecy masking, timemasking, catuion noise, augmentations 
    #self.transforms = nn.Sequential(torchaudio.transforms.FrequencyMasking(freq_mask_param=15),torchaudio.transforms.TimeMasking(time_mask_param=15))
    self.convolution_embedding = nn.Sequential(nn.Conv1d(in_channels = 40, out_channels = 256, kernel_size=3, bias = False, padding='same'), nn.ReLU(), nn.BatchNorm1d(256),nn.Conv1d(in_channels = 256, out_channels = 256, kernel_size=3, bias = False, padding='same'), nn.ReLU(), nn.BatchNorm1d(256),nn.Dropout(p=dropout))
    self.biLSTM_0 = WeightDrop(nn.LSTM(input_size=256, hidden_size=hidden_size, num_layers=num_layers, bidirectional=True, dropout = dropout, batch_first = True),['weight_hh_l0'], dropout = dropout )
    self.biLSTM_1 = WeightDrop(nn.LSTM(input_size=hidden_size*2, hidden_size=hidden_size, num_layers = num_layers, bidirectional=True, dropout = dropout, batch_first = True),['weight_hh_l0'], dropout = dropout )
    self.linear_layers = nn.Sequential(nn.ReLU(), nn.Linear(2*hidden_size, 128, bias=False), nn.ReLU(), torch.nn.Dropout(), nn.Linear(128,42))
  def forward(self, x, lengths_data): 
    # x dimen =  B X L X C
    # CNN dim-Input = (B, C, L), output = (B, C, L)
  
    x = x.permute(0, 2, 1)
    #x = self.transforms(x)
    #Transforms
    x = self.convolution_embedding(x)

    #LSTM Input = B x L X C, Output = B X L X Hidden * D 
    x = x.permute(0,2,1)
    x = torch.nn.utils.rnn.pack_padded_sequence(x, lengths_data, enforce_sorted=False, batch_first=True)

    x, state = self.biLSTM_0(x)
    x, state = self.biLSTM_1(x)
    x, length_1 = torch.nn.utils.rnn.pad_packed_sequence(x, batch_first=True)

    #Linear dim-Iput = * X H, dim-Output = B x L x C
    x = self.linear_layers(x)
    return(x,length_1)

In [ ]:
class LSTMModelResnet(nn.Module):

  def __init__(self, hidden_size, num_layers, dropout):
    super(LSTMModelResnet, self).__init__()
    #frequecy masking, timemasking, catuion noise, augmentations 
    # 10, 10 -- epochs 1-70
    #10, 20 -- epchs 70-78
    #15, 40 - epcohs 78-?

    self.transforms = nn.Sequential(torchaudio.transforms.FrequencyMasking(freq_mask_param=10),torchaudio.transforms.TimeMasking(time_mask_param=20))
    self.convolution_embedding = Resnet18(40)
    self.biLSTM_0 = nn.LSTM(input_size=128, hidden_size=hidden_size, num_layers=num_layers, bidirectional=True, dropout = dropout, batch_first = True)
    self.biLSTM_1 = nn.LSTM(input_size=hidden_size*2, hidden_size=hidden_size, num_layers = num_layers, bidirectional=True, dropout = dropout, batch_first = True)
    self.linear_layers = nn.Sequential(nn.ReLU(), nn.Linear(2*hidden_size, 128, bias=False), nn.ReLU(), torch.nn.Dropout(), nn.Linear(128,42))
    
  def forward(self, x, lengths_data, eval_mode = False): 
    # x dimen =  B X L X C
    # CNN dim-Input = (B, C, L), output = (B, C, L)
    x = x.permute(0, 2, 1)

    if eval_mode == False:
      x = self.transforms(x)

    x = self.convolution_embedding(x)
    #LSTM Input = B x L X C, Output = B X L X Hidden * D 
    x = x.permute(0,2,1)
    x = torch.nn.utils.rnn.pack_padded_sequence(x, lengths_data, enforce_sorted=False, batch_first=True)

    x, state = self.biLSTM_0(x)
    x, state = self.biLSTM_1(x)
    x, length_1 = torch.nn.utils.rnn.pad_packed_sequence(x, batch_first=True)

    #Linear dim-Iput = * X H, dim-Output = B x L x C
    x = self.linear_layers(x)
    return(x,length_1)

In [ ]:
class ResidualBlockResnet34(nn.Module):
  def __init__(self, input_channel_size, keep_dim = True):
    super().__init__()

    #Depending if we are changing our dimensions, we will have to initlize our parameters differently
    if keep_dim == True:  
      output_size = input_channel_size
      self.shortcut_x = nn.Identity()
      self.conv0 = nn.Conv1d(in_channels=input_channel_size, out_channels = output_size, kernel_size=3,stride=1, padding = 'same', bias = False)
    
    else:
      output_size = int(input_channel_size*2)
      self.shortcut_x = nn.Sequential(
      nn.Conv1d(in_channels=input_channel_size, out_channels= output_size, kernel_size = 3, stride = 1, padding = 'same', bias = False),
      nn.BatchNorm1d(output_size))
      self.conv0 = nn.Conv1d(in_channels=input_channel_size, out_channels = output_size, kernel_size=3, stride=1, padding = 'same', bias = False)

    self.bn_0 = nn.BatchNorm1d(output_size)
    self.reLU_0 = nn.ReLU()

    self.conv1 = nn.Conv1d(in_channels=output_size, out_channels = output_size, kernel_size = 3, stride = 1, padding='same', bias = False)
    self.bn_1 = nn.BatchNorm1d(output_size)
    self.reLU_1 = nn.ReLU()

  def forward(self,x):
    shortcut = self.shortcut_x(x)
    
    out = self.conv0(x)
    out = self.bn_0(out)
    out = self.reLU_0(out)
    
    out = self.conv1(out)
    out = self.bn_1(out)
    out = self.reLU_1(out)

    out = out + shortcut

    return(out)

In [ ]:
class Resnet18(nn.Module):
  def __init__(self, in_channels):
    super().__init__()
    self.cnn_layers = nn.Sequential(
        nn.Conv1d(in_channels=in_channels,out_channels=64,kernel_size=3, stride =1, padding='same',bias=False),
        nn.BatchNorm1d(64),
        nn.ReLU(), 
        ResidualBlockResnet34(input_channel_size = 64),
        ResidualBlockResnet34(input_channel_size = 64),
        ResidualBlockResnet34(input_channel_size = 64, keep_dim = False),
        ResidualBlockResnet34(input_channel_size = 128),
        nn.Dropout(p=0.2))
  
  def forward(self, x):
    x = self.cnn_layers(x)
    return(x)


#Model Tools

In [ ]:
def train_model(model, train_loader, optimizer, loss_function, epoch):
  model.train()
  running_loss = 0.0 

  for features, label, features_length, labels_length in train_loader:
    optimizer.zero_grad()
    features, label = features.to(device), label.to(device)

    output, x_lengths = model(features, features_length)
    log_output = torch.nn.functional.log_softmax(output, dim =2) 
    #Permuting from B X L X C to L X B X C
    log_output = log_output.permute(1,0,2)

    loss = loss_function(log_output, label, x_lengths.tolist(), labels_length)
    loss.backward()
    torch.nn.utils.clip_grad_norm(model.parameters(),.5)
    optimizer.step()

    running_loss += loss.item()

    del features
    del label
    del loss
    del log_output
    torch.cuda.empty_cache()

  average_loss = running_loss/len(train_loader.dataset)
  print(f"Epoch:{epoch}, Training Loss:{average_loss}")
  wandb.log({"Training Average Loss":average_loss, 'Epoch':epoch })

In [ ]:
def validate_model(model, val_loader, loss_function, epoch_index, decoder, batch_size):
  model.eval()
  running_loss = 0.0
  running_dist = 0.0
  predictions = []

  with torch.no_grad():
    for features, labels, features_length, labels_length in val_loader: 
      features, labels = features.to(device), labels.to(device)
      
      output, length = model(features, features_length, eval_mode = True)
      log_output = torch.nn.functional.log_softmax(output, dim =2) 
      #Permuting from B X L X C to L X B X C
      log_output = log_output.permute(1,0,2)

      decode_result, decoded_label = decode_full_batch(decoder, log_output, length, labels_length, labels)
      running_dist += lev_distance_batch(decode_result, decoded_label) 

      loss = loss_function(log_output, labels, length.tolist(), labels_length)
      running_loss += loss.item()

      del features
      del labels
      del loss
      del log_output
      torch.cuda.empty_cache()
      
  average_loss = running_loss/len(val_loader.dataset)  
  average_dist = running_dist/len(val_loader.dataset)
  print(f"Epoch:{epoch_index}, Validation Loss:{average_loss}, Validation Levenshtein Distance: {average_dist}")
  wandb.log({"Validation Average Loss":average_loss, "Validation Levenshtein Distance":average_dist })
  return(average_loss)

In [ ]:
def inference(model, test_loader, decoder, batch_size, save_name):
  full_predictions_list = []
  model.eval()
  with torch.no_grad():
    for features, features_length in test_loader:
      features = features.to(device)
      output, length = model(features, features_length, eval_mode =True)
      log_output = torch.nn.functional.log_softmax(output, dim =2)
      #Permuting from B X L X C to L X B X C
      log_output = log_output.permute(1,0,2)

      decode_result, _ = decode_full_batch(decoder, log_output, length)
      full_predictions_list.extend(decode_result)
  
  df = write_model_predictions(full_predictions_list, save_name)
  return(df)

In [ ]:
  def save_model( model, optimizer, path, scheduler, epoch):
    path = path + str(epoch) + '.pt'
    torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'scheduler_state_dict' : scheduler.state_dict(),
    }, path)

In [ ]:
def write_model_predictions(predictions,save_name):
  df = pd.DataFrame(predictions)
  df.columns = ['label']
  df['label'] = df['label']
  df.index.name = 'id'
  df.to_csv('/content/gdrive/MyDrive/IDL/HW/HW3/'+save_name+ '.csv', index = True)
  return(df)

In [ ]:
def deploy_model(model, epoch_total, train_loader, optimizer, loss_function, validation_loader, decoder, batch_size, scheduler, path):
  for epoch_index in range(epoch_total):
    train_model(model, train_loader, optimizer, loss_function, epoch_index)
    validation_average_loss = validate_model(model,validation_loader,loss_function,epoch_index,decoder,batch_size)
    scheduler.step(validation_average_loss)
    save_model(model, optimizer, path, scheduler, epoch_index)

In [ ]:
def load_model_for_inference(save_path, model):
    checkpoint = torch.load(save_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    return model

In [ ]:
def load_model_for_more_training(save_path, model, optimizer, schedular):
    checkpoint = torch.load(save_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    schedular.load_state_dict(checkpoint['scheduler_state_dict'])
    return model, optimizer, schedular

Below functions are used for ensembeling 

In [ ]:
class Ensemble():
  
  def __init__(self):
    pass
  
  #saves the probability vectors of each model
  def save_distributions(self, model_list, test_loader, save_name, log_interpolation = False):
    for index, model in enumerate(model_list):
      full_predictions_list = []
      batch_lengths = []
      model.to(device)
      model.eval()
      with torch.no_grad():
        for features, features_length in test_loader:
          features = features.to(device)
          output, length = model(features, features_length)

          if log_interpolation == False:
            probability_vectors = torch.nn.functional.softmax(output, dim = 2)
          else:
            probability_vectors = torch.nn.functional.log_softmax(output, dim = 2)
          #Dim - List of [B X L X Phonemes]
          full_predictions_list.append(probability_vectors.detach().cpu().numpy())
          #No need to run this more then once 
          if index == len(model_list) - 1:
            batch_lengths.append(features_length)

      full_predictions_list = np.asarray(full_predictions_list, dtype=object)
      save_name += str(index)

      with open(save_name, 'wb') as f:
        np.save(f, full_predictions_list)
    
    batch_lengths = np.asarray(batch_lengths)
    with open('/content/gdrive/MyDrive/IDL/HW/HW3/batch_lengths/lengths.npy', 'wb') as f:
      np.save(f, batch_lengths)

  #perform linear or log interpolation ensembling. 
  def interpolation(self, prob_directory, log_interpolation = False):
    probabilities_dir = os.listdir(prob_directory)
    probabilities_paths = [ prob_directory+'/'+file_name for file_name in probabilities_dir]
    prob_0 = np.load(probabilities_paths[0],allow_pickle=True)    
    aux_probs = [np.load(probabilities_paths[i],allow_pickle=True) for i in range(1, len(probabilities_paths))]

    for minibatch_index in range(len(prob_0)):
      for aux_prob in aux_probs:
        prob_0[minibatch_index] += aux_prob[minibatch_index]
      prob_0[minibatch_index] = prob_0[minibatch_index]/(len(aux_probs)+1)
      prob_0[minibatch_index] = torch.from_numpy(prob_0[minibatch_index])
      if log_interpolation == True:
        prob_0[minibatch_index] = torch.nn.functional.softmax(prob_0[minibatch_index], dim = 2)
    return(prob_0)
  
  #Writes out the ensembled probabilities to a df for kaggle submission
  def ensemble_inference(self, ensembled_probabilities, batch_lengths, decoder, save_name):
    full_predictions_list = []
    ensembled_output = zip(ensembled_probabilities,batch_lengths)
    for probability_vectors, minibatch_length in ensembled_output:
      probability_vectors = probability_vectors.permute(1,0,2)
      #L x B X Probilities
      minibatch_length = torch.from_numpy(np.asarray(minibatch_length))
      decode_results, _ = decode_full_batch(decoder,probability_vectors,minibatch_length)
      full_predictions_list.extend(decode_results)
    df = write_model_predictions(full_predictions_list, save_name)


#Decoding Tools

In [ ]:
N_PHONEMES = 41
PHONEME_LIST = [
    " ",
    "SIL",
    "SPN",
    "AA",
    "AE",
    "AH",
    "AO",
    "AW",
    "AY",
    "B",
    "CH",
    "D",
    "DH",
    "EH",
    "ER",
    "EY",
    "F",
    "G",
    "H",
    "IH",
    "IY",
    "JH",
    "K",
    "L",
    "M",
    "N",
    "NG",
    "OW",
    "OY",
    "P",
    "R",
    "S",
    "SH",
    "T",
    "TH",
    "UH",
    "UW",
    "V",
    "W",
    "Y",
    "Z",
    "ZH"
]

PHONEME_MAP = [
    " ",
    ".", #SIL
    "!", #SPN
    "a", #AA
    "A", #AE
    "h", #AH
    "o", #AO
    "w", #AW
    "y", #AY
    "b", #B
    "c", #CH
    "d", #D
    "D", #DH
    "e", #EH
    "r", #ER
    "E", #EY
    "f", #F
    "g", #G
    "H", #H
    "i", #IH 
    "I", #IY
    "j", #JH
    "k", #K
    "l", #L
    "m", #M
    "n", #N
    "N", #NG
    "O", #OW
    "Y", #OY
    "p", #P 
    "R", #R
    "s", #S
    "S", #SH
    "t", #T
    "T", #TH
    "u", #UH
    "U", #UW
    "v", #V
    "W", #W
    "?", #Y
    "z", #Z
    "Z" #ZH
]

#This is used to map from integer to letter
int_indexes = np.arange(len(PHONEME_MAP))
int_to_phoneme_dict = dict()

for index in int_indexes:
  int_to_phoneme_dict[index] = PHONEME_MAP[index]

In [ ]:
def decode(decoder, output, length, labels):
  #output is given as L X B X C, ctc_decoder takes B X L X C
  output = output.permute(1,0,2)

  #Pass in lenght sequence length 
  beam_results, beam_scores, timesteps, out_len = decoder.decode(output, length)

  #This is for one batch
  results = beam_results[0][0][:out_len[0][0]].cpu().numpy()

  #ADD A FOR FOR BATCHES
  decoded_prediction = [PHONEME_MAP[i] for i in results]
  decoded_labels = [PHONEME_MAP[i] for i in labels[0]]

  #strip padding rstrip 
  
  #Returns the decoded string
  decoded_prediction = ''.join(decoded_prediction)
  decoded_labels = ''.join(decoded_labels)
  return(decoded_prediction, decoded_labels)


In [ ]:
def decode_full_batch(decoder, output, length, labels_length=None, labels=None):
  #output is given as L X B X C, ctc_decoder takes B X L X C
  output = output.permute(1,0,2)

  #Pass in lenght sequence length 
  beam_results, beam_scores, timesteps, out_len = decoder.decode(output, length)

  decoded_prediction_list = []
  decoded_labels_list = []

  #print("beam size is", beam_results.shape)
  batch_size = beam_results.shape[0]

  for b in range(batch_size):
    
    results = beam_results[b][0][:out_len[b][0]].cpu().numpy()
    decoded_prediction = [PHONEME_MAP[i] for i in results]
    decoded_prediction = ''.join(decoded_prediction)
    decoded_prediction_list.append(decoded_prediction)

    if labels != None:
      decoded_labels = [PHONEME_MAP[i] for i in labels[b, :labels_length[b]]]
      decoded_labels = ''.join(decoded_labels)
      decoded_labels_list.append(decoded_labels)
    
#Check that we actually need to decode labels
  return(decoded_prediction_list, decoded_labels_list)

In [ ]:
def lev_distance_batch(list_of_predictions, list_of_labels):
  dist = 0
  for index, word in enumerate(list_of_predictions):
    dist += distance(word,list_of_labels[index])
  return(dist)

#Training Model

In [ ]:
config = {
    'epochs' : 30,
    'lr' : .0006,
    'optimizer' : 'adam',
    'batch_size' : 48,
    'schedular' : 'ReduceLROnPlateau',
    'weight_decay' : 5e-6,
    'LSTM_hidden' : 512,
    'LSTM_layers' : 2,
    'dropout': .25,
    'patience' : 5,
    'factor' : .2,
    'embedding' : 'None',
    'beam_width' : 1,
    'run_name' : 'run17/continued'
}

In [ ]:
path_to_toy_train_data = '/content/gdrive/MyDrive/IDL_HW/HW3/Resources/train_data_toy.npy'
path_to_toy_train_labels = '/content/gdrive/MyDrive/IDL_HW/HW3/Resources/train_labels_toy.npy'
path_to_toy_dev_data = '/content/gdrive/MyDrive/IDL_HW/HW3/Resources/dev_data_toy.npy'
path_to_toy_dev_labels = '/content/gdrive/MyDrive/IDL_HW/HW3/Resources/dev_labels_toy.npy'

path_to_train_data = '/content/HW3P2_Data/train.npy'
path_to_train_labels = '/content/HW3P2_Data/train_labels.npy'
path_to_dev_data = '/content/HW3P2_Data/dev.npy'
path_to_dev_labels = '/content/HW3P2_Data/dev_labels.npy'


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device is on",device)

Device is on cuda:0


In [ ]:
decoder = CTCBeamDecoder(
      PHONEME_MAP,
      beam_width = config['beam_width'],
      blank_id = 0,
      log_probs_input = True)

In [ ]:
rnn = LSTMModelResnet(config['LSTM_hidden'], config['LSTM_layers'], config['dropout'])
rnn = rnn.to(device)

In [ ]:
save_path = '/content/gdrive/MyDrive/IDL/HW/HW3/model_saves/' + config['run_name']

train_dataset = UtteranceDatset(path_to_train_data, path_to_train_labels)
train_dataloader = create_data_loader(train_dataset, shuffle=True, batch_size=config['batch_size'], collate_fn = UtteranceDatset.collate_fn, num_workers=1)
val_dataset = UtteranceDatset(path_to_dev_data,path_to_dev_labels)
val_dataloader = create_data_loader(val_dataset, shuffle=True, batch_size=config['batch_size'], collate_fn = UtteranceDatset.collate_fn, num_workers=1)

#optimizer = torch.optim.Adam(rnn.parameters(), lr = config['lr'], weight_decay=config['weight_decay'])
optimizer = NTASGD(rnn.parameters())
ctc_loss = torch.nn.CTCLoss() 
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=config['patience'], factor=config['factor'])


Run the below line if you want to train from a previous save

In [ ]:
model_path = '/content/gdrive/MyDrive/IDL/HW/HW3/model_saves/run17/continued29.pt'
rnn, optimizer, scheduler = load_model_for_more_training(model_path, rnn, optimizer,scheduler)

Else just run this block

In [ ]:
wandb.init(project="UtteranceToPhoneme", config=config)
deploy_model(rnn, config['epochs'], train_dataloader, optimizer, ctc_loss, val_dataloader, decoder, config['batch_size'], scheduler, save_path )
wandb.finish()

#Inference

Run the below line if you want to run from a save

In [ ]:
#If we want to load a different model
save_path = '/content/gdrive/MyDrive/IDL/HW/HW3/model_saves/run16/continuedd19.pt'

#MAKE SURE TO CHANGE MODEL HERE depenending on the run
rnn = LSTMModelResnet(config['LSTM_hidden'], config['LSTM_layers'], config['dropout'])
rnn = rnn.to(device)
rnn = load_model_for_inference(save_path, rnn)

In [ ]:
path_to_test_data = '/content/HW3P2_Data/test.npy'
test_dataset = UtteranceDatset_Test(path_to_test_data)
test_loader = create_data_loader(test_dataset, shuffle=False, batch_size=config['batch_size'], collate_fn = UtteranceDatset_Test.collate_fn, num_workers=1)

In [ ]:
df=inference(rnn,test_loader,decoder,config['batch_size'], 'predictions/output90')

#Ensembling

Run the below to ensemble models

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device is on",device)
decoder = CTCBeamDecoder(
      PHONEME_MAP,
      beam_width = config['beam_width'],
      blank_id = 0,
      log_probs_input = False)

Device is on cuda:0


In [ ]:
model_path_0 = '/content/gdrive/MyDrive/IDL/HW/HW3/model_saves/run6/52.pt'
rnn_0 = LSTMModelConv(config['LSTM_hidden'], config['LSTM_layers'], config['dropout'])
rnn_0 = load_model_for_inference(model_path_0, rnn_0)

#model_path_1 = '/content/gdrive/MyDrive/IDL/HW/HW3/model_saves/run8/51.pt'
#rnn_1 = pBLSTM(config['LSTM_hidden'], config['LSTM_layers'], config['dropout'])
#rnn_1 = load_model_for_inference(model_path_1, rnn_1)

model_path_2 = '/content/gdrive/MyDrive/IDL/HW/HW3/model_saves/run3/46.pt'
rnn_2 = LSTMModelVanilla(config['LSTM_hidden'], config['LSTM_layers'], config['dropout'])
rnn_2 = load_model_for_inference(model_path_2, rnn_2)

model_path_3 = '/content/gdrive/MyDrive/IDL/HW/HW3/model_saves/run12/continued29.pt'
rnn_3 = LSTMModelConvDropConnect(config['LSTM_hidden'], config['LSTM_layers'], config['dropout'])
rnn_3 = load_model_for_inference(model_path_3, rnn_3)

model_path_4 = '/content/gdrive/MyDrive/IDL/HW/HW3/model_saves/run13/continued42.pt'
rnn_4 = LSTMModelConv(config['LSTM_hidden'], config['LSTM_layers'], config['dropout'])
rnn_4 = load_model_for_inference(model_path_4, rnn_4)

#DEBUGING TO SEE IF THIS ACTUALLY WORKS
#model_list = [rnn_0,rnn_0,rnn_0,rnn_0]
model_list = [rnn_0,rnn_2,rnn_3,rnn_4]

Use this to create probability vectors of each model

In [ ]:
save_path = '/content/gdrive/MyDrive/IDL/HW/HW3/probabilities/probsmodel.npy'
ensembler = Ensemble()
ensembler.save_distributions(model_list,test_loader,save_path,log_interpolation = False)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:695: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at  ../aten/src/ATen/native/cudnn/RNN.cpp:925.)
  self.num_layers, self.dropout, self.training, self.bidirectional)
/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


Use this to combine probability vectors into one distribution

In [ ]:
ensembled_probabilities = ensembler.interpolation('/content/gdrive/MyDrive/IDL/HW/HW3/probabilities',log_interpolation = False)

Use this to run inference

In [ ]:
batch_lengths = np.load('/content/gdrive/MyDrive/IDL/HW/HW3/batch_lengths/lengths.npy', allow_pickle = True)
save_path = 'predictions/myoutt51'
ensembler.ensemble_inference(ensembled_probabilities, batch_lengths, decoder, save_path)